### <font color = 'bisque'> Import packages

In [1]:
# import packages
from selenium import webdriver 
from selenium.webdriver.common.by import By 
from bs4 import BeautifulSoup as bs
import pandas as pd

In [2]:
# instantiate a Chrome options object
options = webdriver.ChromeOptions() 
# set the options to use Chrome in headless mode (used for running the script in the background)
options.add_argument("--headless=new") 
# initialize an instance of the Chrome driver (browser) in headless mode
driver = webdriver.Chrome(options=options)

### <font color  = 'bisque'> Collect data using selenium and store it in a database

In [4]:
# install sqlite3 
#!pip install db-sqlite3

In [3]:
# import sqlite3
import sqlite3
# Create a connection with a database (renting_rooms.db)
conn = sqlite3.connect('renting_rooms.db')
# create a cursor (allows to interact with the database)
c = conn.cursor()
# create a table
c.execute('''CREATE TABLE RR_table(details, price, adress)''')

In [4]:
# Scrape data over multiples pages
for p in range(1, 10):
    # url
    url = f'https://ci.coinafrique.com/categorie/appartements?page={p}'
    # get the url 
    driver.get(url)
    # get the content 
    soup = bs(driver.page_source, 'html.parser')
    # find containers 
    containers = soup.find_all('div', 'col s6 m4 l3')
    for container in containers:
        try: 
            # scrape the details 
            details = container.find('p', class_ = 'ad__card-description').text 
            # scrape the price 
            price = container.find('p', class_ ='ad__card-price').text 
            # scrape the adress
            adress = container.find('p', class_ = 'ad__card-location').span.text 
            # add data to RR_table from renting_rooms.db
            c.execute('''INSERT INTO RR_table VALUES(?,?,?)''',(details, price, adress))
            # commit the request 
            conn.commit()
        except:
            pass

In [6]:
# import the table from renting_rooms.db
df = pd.read_sql_query('select * from RR_table', conn)

In [7]:
# Show the dataframe
df.head()

,details,price,adress
0,Location appartement 3 pièces à Cocody Rivi...,350 000CFA,"Cocody, Abidjan, Côte d'Ivoire"
1,Location appartement 4 pièces aux Vallons,850 000CFA,"Cocody, Abidjan, Côte d'Ivoire"
2,Location appartement 2 chambres Riviera Bonoumin,600 000CFA,"Cocody, Abidjan, Côte d'Ivoire"
3,Location appartement 3 pièces à Marcory Zone 4,700 000CFA,"Marcory, Abidjan, Côte d'Ivoire"
4,Location appartement 4 pièces à Cocody 2 Plat...,700 000CFA,"Cocody, Abidjan, Côte d'Ivoire"


In [8]:
# Dimension df
df.shape

(756, 3)

In [ ]:
url = 'https://ci.coinafrique.com/categorie/appartements?page=13'
# scrape data (number of rooms, number of bathroom, area, price, location, image_link) over 200 pages 
# using selenium combined with BeautifulSoup
# Clean the data
# Add the data table to a database that you have created